In [ ]:
# Open in Colab and click the link
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
! git clone https://github.com/youzhanghe123/Lora.git
%cd Lora

Cloning into 'Lora'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 35 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 15.41 KiB | 15.41 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/Lora


In [ ]:
import sys
sys.path.append('.')  # This makes 'src' importable

In [ ]:
!ls -Lora
!ls -Lora src/

total 28
drwxr-xr-x 2 root 4096 Oct 30 04:41 src
-rw-r--r-- 1 root  436 Oct 30 04:27 run.sh
-rw-r--r-- 1 root   40 Oct 30 04:27 requirements.txt
-rw-r--r-- 1 root 1082 Oct 30 04:27 main.py
drwxr-xr-x 8 root 4096 Oct 30 04:27 .git
drwxr-xr-x 1 root 4096 Oct 30 04:27 ..
drwxr-xr-x 4 root 4096 Oct 30 04:27 .
total 32
-rw-r--r-- 1 root 6566 Oct 30 04:27 trainer.py
-rw-r--r-- 1 root 9577 Oct 30 04:41 lora.py
-rw-r--r-- 1 root 3857 Oct 30 04:41 dataset.py
drwxr-xr-x 4 root 4096 Oct 30 04:27 ..
drwxr-xr-x 2 root 4096 Oct 30 04:41 .


In [ ]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00


In [ ]:
import datasets

In [ ]:
device = "cuda"

In [ ]:
# All-in-one training script
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from src.lora import add_lora_to_model
from src.dataset import load_and_prepare_data
from src.trainer import LoRATrainer
import os

# Setup
output_dir = "/content/drive/MyDrive/lora_outputs"
os.makedirs(output_dir, exist_ok=True)

# Load model
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

# Add LoRA
model, _ = add_lora_to_model(
    model,
    use_attention_lora=True,
    use_ffn_lora=True,

    save_path = output_dir
)
print("yes model lora")
# Prepare data
train_dataloader, test_encodings = load_and_prepare_data(
    tokenizer,
    batch_size=64,
    chunk_size=256,
    subset_fraction=0.1  # Use 0.1 for testing, 1.0 for full training
)
print("yes data lora")
# Train
trainer = LoRATrainer(
    model=model,
    train_dataloader=train_dataloader,
    test_encodings=test_encodings,
    save_dir=output_dir,
    device = device
)
trainer.train()
print("yes train lora")
# Save
save_path = os.path.join(output_dir, 'final_lora_weights.pt')
torch.save({
    'lora_state': model.state_dict(),
    'final_perplexity': trainer.best_perplexity
}, save_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Created 5752 chunks
Loading test split of WikiText...
Created 6853 chunks
yes data lora


Evaluating: 100%|█████████▉| 1120/1124 [00:31<00:00, 35.52it/s]


yes train lora


In [ ]:

# Read the weights file


# Path to your .pt file
weights_path = "/content/drive/MyDrive/lora_outputs/final_lora_weights.pt"

def analyze_lora_weights(weights_path: str):
    """
    Perform detailed analysis of LoRA weights
    """
    checkpoint = torch.load(weights_path, map_location='cpu')

    if 'lora_state' not in checkpoint:
        print("No LoRA state found in checkpoint")
        return
    print(checkpoint.keys())
    lora_state = checkpoint['lora_state']

    print(lora_state.keys())

    # Analyze weight statistics
    print("\n=== Weight Statistics ===")



    return lora_state



In [ ]:
lora_state = analyze_lora_weights(weights_path)

<ipython-input-29-03eb69cefa18>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_path, map_location='cpu')


dict_keys(['lora_state', 'final_perplexity'])
odict_keys(['model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'model.decoder.final_layer_norm.weight', 'model.decoder.final_layer_norm.bias', 'model.decoder.layers.0.self_attn.k_proj.lora_A', 'model.decoder.layers.0.self_attn.k_proj.lora_B', 'model.decoder.layers.0.self_attn.k_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.k_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.v_proj.lora_A', 'model.decoder.layers.0.self_attn.v_proj.lora_B', 'model.decoder.layers.0.self_attn.v_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.v_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.q_proj.lora_A', 'model.decoder.layers.0.self_attn.q_proj.lora_B', 'model.decoder.layers.0.self_attn.q_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.q_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.out_proj.weight', 'model.decoder.layers.0.self_attn.out_proj.bias',

In [ ]:
lora_state.keys()

odict_keys(['model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'model.decoder.final_layer_norm.weight', 'model.decoder.final_layer_norm.bias', 'model.decoder.layers.0.self_attn.k_proj.lora_A', 'model.decoder.layers.0.self_attn.k_proj.lora_B', 'model.decoder.layers.0.self_attn.k_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.k_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.v_proj.lora_A', 'model.decoder.layers.0.self_attn.v_proj.lora_B', 'model.decoder.layers.0.self_attn.v_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.v_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.q_proj.lora_A', 'model.decoder.layers.0.self_attn.q_proj.lora_B', 'model.decoder.layers.0.self_attn.q_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.q_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.out_proj.weight', 'model.decoder.layers.0.self_attn.out_proj.bias', 'model.decoder.layers.0.self_attn_layer_norm.

In [ ]:
type(lora_state["model.decoder.layers.0.self_attn.k_proj.lora_A"])

torch.Tensor

In [ ]:
lora_state["model.decoder.layers.0.self_attn.k_proj.lora_A"].shape

torch.Size([8, 768])

In [ ]:
lora_state["model.decoder.layers.0.self_attn.k_proj.lora_B"].shape

torch.Size([768, 8])

In [ ]:
lora_state["model.decoder.layers.0.self_attn.k_proj.existing_layer.weight"].shape

torch.Size([768, 768])